## 1. Data Preprocessing

In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv('Reviews.csv')

# Keep only relevant columns
df = df[['Score', 'Text']].copy()

# Drop rows with missing text
df.dropna(subset=['Text'], inplace=True)

# Convert Score to Sentiment
def map_score_to_sentiment(score):
    if score <= 2:
        return 'negative'
    elif score == 3:
        return 'neutral'
    else:
        return 'positive'

df['Sentiment'] = df['Score'].apply(map_score_to_sentiment)

## 2. Feature Extraction

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Split the dataset
X = df['Text']
y = df['Sentiment']

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Use TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

## 3. Model Selection

### Lexicon 

In [6]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Sample 500 reviews for lexicon analysis
df_sample = df.sample(n=500, random_state=42)
true_labels = df_sample['Sentiment'].tolist()

# Initialize
analyzer = SentimentIntensityAnalyzer()
textblob_preds, vader_preds = [], []

for text in df_sample['Text']:
    # TextBlob
    tb_score = TextBlob(text).sentiment.polarity
    textblob_preds.append('positive' if tb_score > 0 else 'negative' if tb_score < 0 else 'neutral')
    
    # VADER
    vader_score = analyzer.polarity_scores(text)['compound']
    vader_preds.append('positive' if vader_score > 0.05 else 'negative' if vader_score < -0.05 else 'neutral')

### Machine Learning

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# Train Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Train Logistic Regression
lr_model = LogisticRegression(max_iter=2000, solver='saga')
lr_model.fit(X_train, y_train)

LogisticRegression(max_iter=2000, solver='saga')

## 4. Model Evaluation

In [8]:
from sklearn.metrics import classification_report

# Lexicon-based
print("TextBlob Classification Report:\n")
print(classification_report(true_labels, textblob_preds, target_names=['negative', 'neutral', 'positive']))

print("\nVADER Classification Report:\n")
print(classification_report(true_labels, vader_preds, target_names=['negative', 'neutral', 'positive']))

# ML-based
nb_preds = nb_model.predict(X_test)
lr_preds = lr_model.predict(X_test)

print("\nNaive Bayes Classification Report:\n")
print(classification_report(y_test, nb_preds, target_names=['negative', 'neutral', 'positive']))

print("\nLogistic Regression Classification Report:\n")
print(classification_report(y_test, lr_preds, target_names=['negative', 'neutral', 'positive']))

TextBlob Classification Report:

              precision    recall  f1-score   support

    negative       0.52      0.38      0.44        74
     neutral       0.00      0.00      0.00        29
    positive       0.85      0.94      0.89       397

    accuracy                           0.80       500
   macro avg       0.45      0.44      0.44       500
weighted avg       0.75      0.80      0.77       500


VADER Classification Report:

              precision    recall  f1-score   support

    negative       0.67      0.35      0.46        74
     neutral       0.00      0.00      0.00        29
    positive       0.84      0.96      0.90       397

    accuracy                           0.81       500
   macro avg       0.50      0.44      0.45       500
weighted avg       0.77      0.81      0.78       500


Naive Bayes Classification Report:

              precision    recall  f1-score   support

    negative       0.83      0.28      0.41     24278
     neutral       0.42     

**By Addelina binti Mohd Zulkifli (SW01082366) and 
     Mysara Qistina binti Mahadzir (SW01083524)**

#### TextBlob (Lexicon-Based)  
Strengths:
- Very easy to use and requires no training.
- Good for simple, small-scale sentiment analysis.
- Provides both polarity and subjectivity scores.
  
Weaknesses:  
- Limited vocabulary and rules.
- Doesn’t handle sarcasm, context, or domain-specific language well.
- Often struggles with detecting neutral sentiment accurately.

#### VADER (Lexicon-Based)  
Strengths:
- Specifically designed for sentiment in social media text (handles emojis, capitalization, slang).
- Better at understanding intensity of sentiment.
- Fast and lightweight.
  
Weaknesses:
- Still rule-based, so it misses deeper meaning or sarcasm.
- Not suitable for complex or domain-specific texts.
- May misclassify longer or mixed-opinion reviews.

#### Multinomial Naive Bayes (Machine Learning-Based)  
Strengths:
- Very fast and efficient for large text datasets.
- Performs well with clearly separated categories.
- Easy to implement and interpret.
  
Weaknesses:
- Assumes features are independent, which is not always true in language.
- Struggles with classes that are underrepresented (like neutral).
- Doesn’t model word order or context.

#### Logistic Regression (Machine Learning-Based)  
Strengths:
- Strong and reliable performance for classification tasks.
- Works well with TF-IDF features.
- Can handle imbalanced datasets better than Naive Bayes with proper tuning.
  
Weaknesses:
- Slower training compared to Naive Bayes.
- Doesn’t capture word order or relationships like more advanced models (e.g., RNNs or Transformers).
- Needs more data and preprocessing compared to lexicon models.aset.
